In [ ]:
import pathlib

from compas.datastructures import Mesh
from compas.files import OBJ

# =============================================================================
# Data
# =============================================================================

FILE = pathlib.Path().cwd().parent / "data" / "crossvault.obj"

obj = OBJ(FILE)
obj.read()

meshes = []
for name in obj.objects:
    vertices, faces = obj.objects[name]
    mesh: Mesh = Mesh.from_vertices_and_faces(vertices, faces)
    mesh.scale(0.025, 0.025, 0.025)
    mesh.name = name
    meshes.append(mesh)


In [ ]:
from compas_dem.elements import BlockElement
from compas_dem.models import BlockModel

model = BlockModel.from_boxes(meshes)

In [ ]:
model.compute_contacts(tolerance=1e-3, minimum_area=1e-2)

element: BlockElement

for element in model.elements():
    if model.graph.degree(element.graphnode) == 1:
        element.is_support = True

In [ ]:
from compas.colors import Color
from compas_notebook import Viewer

color_support = Color.red()
color_contact = Color.green()

viewer = Viewer()

for element in model.supports():
    viewer.scene.add(
        element.modelgeometry,
        show_faces=True,
        show_edges=True,
        facecolor=color_support,
        edgecolor=color_support.contrast,
    )

for element in model.blocks():
    viewer.scene.add(
        element.modelgeometry,
        show_faces=False,
        show_edges=True,
        facecolor=None,
        edgecolor=Color.black()
)

for contact in model.contacts():
    viewer.scene.add(
        contact.polygon,
        facecolor=color_contact,
        linecolor=color_contact.contrast,
        show_edges=False,
    )

viewer.show()